https://www.mfapi.in/

In [1]:
import requests 
from bs4 import BeautifulSoup 
import urllib, json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [2]:
class MF:
    def __init__(self,url):
        response= urllib.request.urlopen(url)
        summary = json.loads(response.read())
        self.FundHouse=summary['meta']['fund_house']
        self.SchemeType=summary['meta']['scheme_type']
        self.SchemeCategory=summary['meta']['scheme_category']
        self.SchemeName=summary['meta']['scheme_name']
        self.data=summary['data']
        
def fill_missing_dates(df):
    result = json_normalize(df)
    result['date'] = pd.to_datetime(result['date'],format='%d-%m-%Y') #Converting date column from str to datetime
    result['Invested_Val']=0
    result['Units']=0
    result=result.sort_values(by=['date']) 
    start_date='01-01-2016'
    end_date='01-07-2021'
    mask = (result['date'] > start_date) & (result['date'] <= end_date)
    result =result.loc[mask]
     #Setting date as index
    result=result.set_index('date')
    idx = pd.date_range(start_date,end_date)
    result=result.reindex(idx) #Reindexing dates so that missing values are filled 
    result.fillna(method='bfill',inplace=True)
    result=result.reset_index()
    return result
    
def units_bought(dfg,val):
    i=0
    dfg['nav'] = dfg['nav'].astype(float)
    while i<len(dfg.index):
        now=dfg['index'][i]
        if now.day==1:
            dfg['Invested_Val'][i]=val
        i=i+1
    dfg['Units'] = (dfg['Invested_Val'])/(dfg['nav'])    
    return dfg

def total_return(df):
    nav1=df['nav'].iloc[-1]
    units=df['Units'].sum()
    ret=units*nav1
    return(ret)

def cumulative_val(df):
    df['CumulativeUnits']=df['Units']
    df['CumulativeUnits']=df['CumulativeUnits'].cumsum()
    df['CumulInvest']=df['Invested_Val']
    df['CumulInvest']=df['CumulInvest'].cumsum()
    df['CumulativeValue'] = df.CumulativeUnits * df.nav
    return(df)

SyntaxError: invalid syntax (<ipython-input-2-8256cb5ca4fe>, line 3)

In [198]:
url_dict={'https://api.mfapi.in/mf/120465':3000,
          
}

In [199]:
dfg1=pd.DataFrame(columns=['FundName','Total_Invested','Latest_NAV','Current_Val'])
for url in url_dict:
    mf=MF(url)
    data=fill_missing_dates(mf.data)
    unit=units_bought(data,val=url_dict[url])
    tot_lc=total_return(unit)
    add_row={'FundName':mf.SchemeName,
    'Total_Invested':unit['Invested_Val'].sum(),
    'Latest_NAV':unit['nav'].iloc[-1],
    'Current_Val':tot_lc}
    dfg1=dfg1.append(add_row,ignore_index=True)
    dfg1['Category']='Portfolio'
dfg1

,FundName,Total_Invested,Latest_NAV,Current_Val,Category
0,HDFC Index Fund-Nifty 50 Plan-Growth Plan,183000.0,129.3623,263170.786636,Portfolio


In [200]:
index_dict={'https://api.mfapi.in/mf/119788':3000
            }

In [201]:
dfg2=pd.DataFrame(columns=['FundName','Total_Invested','Latest_NAV','Current_Val'])
for url in index_dict:
    mf=MF(url)
    data=fill_missing_dates(mf.data)
    unit=units_bought(data,val=index_dict[url])
    tot_lc=total_return(unit)
    add_row={'FundName':mf.SchemeName,
    'Total_Invested':unit['Invested_Val'].sum(),
    'Latest_NAV':unit['nav'].iloc[-1],
    'Current_Val':tot_lc}
    dfg2=dfg2.append(add_row,ignore_index=True)
    dfg2['Category']='Index'
dfg2

,FundName,Total_Invested,Latest_NAV,Current_Val,Category
0,SBI GOLD FUND- DIRECT PLAN - GROWTH,183000.0,16.3443,277013.923541,Index


In [202]:
df_list = [dfg1[['Category','FundName','Current_Val']],
            dfg2[['Category','FundName','Current_Val']]
            ]
dfg3=pd.concat(df_list, ignore_index=True)

In [203]:
from plotly.offline import iplot, init_notebook_mode
import numpy as np
import matplotlib.pyplot as plt

In [204]:
import plotly.express as px
fig = px.bar(dfg3, x="Category", y="Current_Val", color="FundName")
fig.show()

In [205]:
dfc1=pd.DataFrame()
for url in url_dict:
    mf=MF(url)
    data=fill_missing_dates(mf.data)
    unit=units_bought(data,val=url_dict[url])
    fund=cumulative_val(unit)
    df_list = [dfc1,fund[['index', 'nav','CumulativeUnits', 'CumulativeValue']]]
    dfc1=pd.concat(df_list, ignore_index=True)
dfc1['Category']='Portfolio'
dfc4=dfc1.groupby([dfc1['index'],'Category']).sum().reset_index()

In [206]:
dfc2=pd.DataFrame()
for url in index_dict:
    mf=MF(url)
    data=fill_missing_dates(mf.data)
    unit=units_bought(data,val=index_dict[url])
    fund=cumulative_val(unit)
    df_list = [dfc2,fund[['index', 'nav','CumulativeUnits', 'CumulativeValue']]]
    dfc2=pd.concat(df_list, ignore_index=True)
dfc2['Category']='Index'

In [207]:
dfc_list = [dfc4[['Category','CumulativeValue','index']],
            dfc2[['Category','CumulativeValue','index']]
            ]
dfc3=pd.concat(dfc_list, ignore_index=True)

In [208]:
fig = px.line(dfc3, x="index", y="CumulativeValue",color='Category')
fig.show()